In [325]:
import pandas as pd
import lightgbm as lgbm
import numpy as np

In [332]:
train=pd.read_csv('./playground-series-s3e11/train.csv')
X_test=pd.read_csv('./playground-series-s3e11/test.csv')
columns={'avg_cars_at home(approx).1': 'avg_cars_at_home', 'store_sales(in millions)': 'store_sales', 'unit_sales(in millions)':'unit_sales'}
train.rename(columns=columns, inplace = True )
X_test.rename(columns=columns, inplace = True )

In [333]:
Y_train = np.log(train['cost'])
X_train = train.drop(columns=['cost'])
X_train.head()

,id,store_sales,unit_sales,total_children,num_children_at_home,avg_cars_at_home,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist
0,0,8.61,3.0,2.0,2.0,2.0,10.30,1.0,0.0,32.0,36509.0,0.0,0.0,0.0,0.0,0.0
1,1,5.00,2.0,4.0,0.0,3.0,6.66,1.0,0.0,1.0,28206.0,1.0,0.0,0.0,0.0,0.0
2,2,14.08,4.0,0.0,0.0,3.0,21.30,1.0,0.0,26.0,21215.0,1.0,0.0,0.0,0.0,0.0
3,3,4.02,3.0,5.0,0.0,0.0,14.80,0.0,1.0,36.0,21215.0,1.0,0.0,0.0,0.0,0.0
4,4,2.13,3.0,5.0,0.0,3.0,17.00,1.0,1.0,20.0,27694.0,1.0,1.0,1.0,1.0,1.0


In [334]:
stores_features= ['coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist']
X_train['store_score'] = X_train[stores_features].sum(axis=1)
X_train['store_score_ratio'] = X_train['store_sqft'] / X_train['store_score']
X_test['store_score'] = X_test[stores_features].sum(axis=1)
X_test['store_score_ratio'] = X_test['store_sqft'] / X_test['store_score']

In [335]:
fnames = ['store_sqft', 'florist', 'salad_bar', 'prepared_food', 'coffee_bar',
          'video_store', 'total_children', 'avg_cars_at_home', 'num_children_at_home',
         'store_score', 'store_score_ratio']

In [336]:
# mymodel = lgbm.LGBMRegressor(num_leaves=124)
mymodel = lgbm.LGBMRegressor(num_leaves=248, metric='rmse')
mymodel.fit(X_train[fnames], Y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027411 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 109
[LightGBM] [Info] Number of data points in the train set: 360336, number of used features: 11
[LightGBM] [Info] Start training from score 4.552303


LGBMRegressor(metric='rmse', num_leaves=248)

In [337]:
Y_predicted = mymodel.predict(X_test[fnames])
Y_predicted = np.exp(Y_predicted);
# Создание DataFrame с предсказаниями
df_predictions = pd.DataFrame({'id': X_test['id'], 'cost': Y_predicted})
# Сохранение предсказаний в CSV файл
df_predictions.to_csv('predictions_nl=248_and_rmse_and_store_score_and_LOFO.csv', index=False)

In [321]:


fnames = ['store_sqft', 'florist', 'salad_bar', 'prepared_food', 'coffee_bar',
          'video_store', 'total_children', 'avg_cars_at_home', 'num_children_at_home',
         'store_score', 'store_score_ratio']
d_train = lgbm.Dataset(X_train[fnames], label=Y_train)
params = {}
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['num_leaves'] = 248
print(params)
N_FOLDS = 3
MAX_BOOST_ROUNDS = 700
cv_results = lgbm.cv(params, d_train, nfold=N_FOLDS, stratified=False)

{'objective': 'regression', 'metric': 'rmse', 'num_leaves': 248}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009091 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 109
[LightGBM] [Info] Number of data points in the train set: 240224, number of used features: 11
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011168 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 109
[LightGBM] [Info] Number of data points in the train set: 240224, number of used features: 11
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032631 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 109
[LightGBM] [Info] Number of data p

In [322]:

print(np.mean(cv_results['valid rmse-mean']))

0.2986422027513529
